In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *
from utils.pdf_functs import *

/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-data-lake


/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Database and directories

In [2]:
db_file_name = 'cev-database-coordinates-v1.db'
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)

## 1) Define parameters to run a `search`
This can be a new search or retake a previous one

In [3]:
search_id = '4bd70bab-b20c-4e27-b514-92ff6fa70351'
#search_id = '010576e8-90f1-42c4-88bc-3965b5c4c238'
search_date = '2024-05-02'
#search_date = '2024-04-21'

## 2) Define a dataframe with download coordinates
The dataframe must contain 1 row per `comuna` meaning 348 rows in total.

In [4]:
limit = 250
query = f"""
        SELECT * FROM evals_pdf_reports_download_coordinates
        WHERE search_id = '4bd70bab-b20c-4e27-b514-92ff6fa70351'
         AND codigo_informe IS NOT NULL
        ORDER BY RANDOM()
        LIMIT {limit};
        """
print(query)


        SELECT * FROM evals_pdf_reports_download_coordinates
        WHERE search_id = '4bd70bab-b20c-4e27-b514-92ff6fa70351'
         AND codigo_informe IS NOT NULL
        ORDER BY RANDOM()
        LIMIT 25;
        


In [5]:
df = create_dataframe_from_query(db_file_path, query)
# Drop specified columns
columns_to_drop = ['id']
df = df.drop(columns=columns_to_drop)
df.head()

,comuna_id,region_id,tipo_evaluacion,eval_number,pagina,eval_number_pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument
0,163,13,2,138559,2,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,dff6eaf1-095c-534a-9131-0395e7e4a08a,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,None,+7UB1yfh7V3Rtl0C0TTZVKLD71zxOhECasZWOAFzRsP9ge...,Page$2
1,128,14,1,78789,10,5,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,01c6dd88-3766-5e3a-a552-2134f7d4c41e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,AF6sMOJgixNEPfo5yBxji0vTfLGcYgJJirUhN+26PNmqNH...,Page$10
2,290,7,2,104914,6,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,657a8fcf-e7d3-5ec9-85d9-6071dfc901b4,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uJA76PuIHteIdnHkgKbJRkrk8K+9o+z3Ik0uCwmxbW3/mn...,Page$6
3,90,13,2,129009,19,8,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,b2e27c10-42c0-57e3-9349-4311181aaee1,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uNoHQqpoe0LGSFzZqwQKoLhLZs5WrQ3x1Q90sAieFuBV0w...,Page$19
4,79,7,1,19415,109,6,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,a638a6ce-d4a5-5c0d-97ba-1f60d3f64a29,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,iq7qcCzMcStxuwDfqyhP74nvO9foMQHKHTH98GU51QERf+...,Page$109


Set additional features to track the status of downloads as well as times and file names

In [6]:
df['status'] = None
df['pdf_filename'] = None
df.head()

,comuna_id,region_id,tipo_evaluacion,eval_number,pagina,eval_number_pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename
0,163,13,2,138559,2,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,dff6eaf1-095c-534a-9131-0395e7e4a08a,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,None,+7UB1yfh7V3Rtl0C0TTZVKLD71zxOhECasZWOAFzRsP9ge...,Page$2,None,None
1,128,14,1,78789,10,5,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,01c6dd88-3766-5e3a-a552-2134f7d4c41e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,AF6sMOJgixNEPfo5yBxji0vTfLGcYgJJirUhN+26PNmqNH...,Page$10,None,None
2,290,7,2,104914,6,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,657a8fcf-e7d3-5ec9-85d9-6071dfc901b4,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uJA76PuIHteIdnHkgKbJRkrk8K+9o+z3Ik0uCwmxbW3/mn...,Page$6,None,None
3,90,13,2,129009,19,8,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,b2e27c10-42c0-57e3-9349-4311181aaee1,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uNoHQqpoe0LGSFzZqwQKoLhLZs5WrQ3x1Q90sAieFuBV0w...,Page$19,None,None
4,79,7,1,19415,109,6,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,a638a6ce-d4a5-5c0d-97ba-1f60d3f64a29,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,iq7qcCzMcStxuwDfqyhP74nvO9foMQHKHTH98GU51QERf+...,Page$109,None,None


## 3) Check first if any pdf files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [7]:
# Define the directory path
pdf_files_dir = os.path.join(project_folder_path, 'data', 'raw', '3_evals_reports', 'pdf_files')

# Check if the directory exists
if not os.path.exists(pdf_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(pdf_files_dir)
    print(f"Directory '{pdf_files_dir}' created.")

# List all files in the directory
pdf_file_paths = find_pdf_files(pdf_files_dir)
print(f'The directory: {pdf_files_dir} contains {len(pdf_file_paths)} out of {df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-data-lake/data/raw/3_evals_reports/pdf_files contains 12405 out of 25


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

In [8]:
# Assuming df is your DataFrame
df['combined_key'] = df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str) + '_' + df['eval_id'].astype(str)


for pdf_file_path in pdf_file_paths:
    pdf_file_name = os.path.split(pdf_file_path)[-1]
    pdf_file_list = pdf_file_name.split('_')

    region_id = int(pdf_file_list[0])
    comuna_id = int(pdf_file_list[1])
    tipo_evaluacion = int(pdf_file_list[2])
    eval_id = pdf_file_list[-1].split('.')[0]

    #row = df[df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str)+ '_' + df['eval_id'].astype(str) == str(region_id) + '_' + str(comuna_id) + '_' + str(tipo_evaluacion) + '_' + str(eval_id)]
    combined_key = f"{region_id}_{comuna_id}_{tipo_evaluacion}_{eval_id}"

    row_index = df.index[df['combined_key'] == combined_key]
    if not row_index.empty:
        df.loc[row_index, 'status'] = 'Successful'
        df.loc[row_index, 'pdf_filename'] = pdf_file_name

# Drop the combined_key column at the end
df.drop(columns=['combined_key'], inplace=True)

df.head()

,comuna_id,region_id,tipo_evaluacion,eval_number,pagina,eval_number_pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename
0,163,13,2,138559,2,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,dff6eaf1-095c-534a-9131-0395e7e4a08a,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,None,+7UB1yfh7V3Rtl0C0TTZVKLD71zxOhECasZWOAFzRsP9ge...,Page$2,None,None
1,128,14,1,78789,10,5,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,01c6dd88-3766-5e3a-a552-2134f7d4c41e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,AF6sMOJgixNEPfo5yBxji0vTfLGcYgJJirUhN+26PNmqNH...,Page$10,None,None
2,290,7,2,104914,6,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,657a8fcf-e7d3-5ec9-85d9-6071dfc901b4,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uJA76PuIHteIdnHkgKbJRkrk8K+9o+z3Ik0uCwmxbW3/mn...,Page$6,None,None
3,90,13,2,129009,19,8,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,b2e27c10-42c0-57e3-9349-4311181aaee1,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uNoHQqpoe0LGSFzZqwQKoLhLZs5WrQ3x1Q90sAieFuBV0w...,Page$19,None,None
4,79,7,1,19415,109,6,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,a638a6ce-d4a5-5c0d-97ba-1f60d3f64a29,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,iq7qcCzMcStxuwDfqyhP74nvO9foMQHKHTH98GU51QERf+...,Page$109,None,None


In [9]:
df['status'].value_counts(dropna=False)

status
None          22
Successful     3
Name: count, dtype: int64

### Download pdf files: 1 file per evaluacion

In [10]:
HOME_URL = 'http://calificacionenergeticaweb.minvu.cl/Publico/BusquedaVivienda.aspx'

In [11]:
df = df[df['status'] != 'Successful'].reset_index(drop=True)
print(f'{df.shape[0]} reports to download')
df.head()

22 reports to download


,comuna_id,region_id,tipo_evaluacion,eval_number,pagina,eval_number_pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename
0,163,13,2,138559,2,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,dff6eaf1-095c-534a-9131-0395e7e4a08a,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,None,+7UB1yfh7V3Rtl0C0TTZVKLD71zxOhECasZWOAFzRsP9ge...,Page$2,None,None
1,128,14,1,78789,10,5,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,01c6dd88-3766-5e3a-a552-2134f7d4c41e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,AF6sMOJgixNEPfo5yBxji0vTfLGcYgJJirUhN+26PNmqNH...,Page$10,None,None
2,290,7,2,104914,6,7,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,657a8fcf-e7d3-5ec9-85d9-6071dfc901b4,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uJA76PuIHteIdnHkgKbJRkrk8K+9o+z3Ik0uCwmxbW3/mn...,Page$6,None,None
3,90,13,2,129009,19,8,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,b2e27c10-42c0-57e3-9349-4311181aaee1,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,uNoHQqpoe0LGSFzZqwQKoLhLZs5WrQ3x1Q90sAieFuBV0w...,Page$19,None,None
4,79,7,1,19415,109,6,4bd70bab-b20c-4e27-b514-92ff6fa70351,2024-05-02,a638a6ce-d4a5-5c0d-97ba-1f60d3f64a29,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,iq7qcCzMcStxuwDfqyhP74nvO9foMQHKHTH98GU51QERf+...,Page$109,None,None


In [12]:
MAX_ATTEMPTS = 5
DELAY_BETWEEN_ATTEMPTS = 15

attempts = 0
while attempts < MAX_ATTEMPTS:
    all_successful = True  # Flag to track if all rows are successful in this attempt
    reports_to_download = df[df['status'] != 'Successful'].shape[0]
    print(f"Attempt {attempts + 1} of {MAX_ATTEMPTS}: {reports_to_download} reports to download")

    for index, row in df.iterrows():
        if df.loc[index, 'status'] != 'Successful':
            region_id = str(row['region_id'])
            comuna_id = str(row['comuna_id'])
            tipo_evaluacion = str(row['tipo_evaluacion'])
            #eventtarget = row['eventtarget']
            eventargument = 'Page$' + str(row['pagina'])
            viewstate = row['viewstate']
            target_report_label_code = row['codigo_informe']
            # eventtarget
            if str(tipo_evaluacion) == '1':
                eventtarget = 'ctl00$ContentPlaceHolder1$grdViviendasPre'
            else:
                eventtarget = 'ctl00$ContentPlaceHolder1$grdViviendasCal'
            
            form_data_pdf = form_data_pdf_report(eventtarget, eventargument, viewstate, region_id, comuna_id, tipo_evaluacion, target_report_label_code)
            
            print(f" * Downloading report {index+1} out of {df.shape[0]}")
           
            try:
                response = requests.post(HOME_URL, data=form_data_pdf)

                if response.status_code == 200:
                    # Record the current time after making the request
                    request_time = datetime.now()
                    downloaded_on = request_time.strftime("%Y-%m-%d")
                    downloaded_at = request_time.strftime("%Y-%m-%d %H:%M:%S")
                    
                    df.loc[index, 'status'] = 'Successful'
                    
                    # Saving pdf data into a local folder
                    # pdf_path = os.path.join(pdf_files_dir, region_id)
                    pdf_path = '/mnt/c/Users/rober/Desktop/test_doawload/'
                   
                    if not os.path.exists(pdf_path):
                        os.makedirs(pdf_path)
                        
                    pdf_filename = region_id + "_" + comuna_id + "_" + tipo_evaluacion + "_" + row['eval_id'] + '.pdf'                    

                    with open(os.path.join(pdf_path, pdf_filename), "wb") as f:
                        f.write(response.content)                    
                    
                    df.loc[index, 'pdf_filename'] = pdf_filename
                    #df.loc[index, 'downloaded_at'] = downloaded_at
                else:
                    df.loc[index, 'status'] = 'Failed'
                    print(f"Request failed for region {region_id}, commune {comuna_id}. Skipping to the next row.")
                    continue  # Skip to the next iteration of the loop
                #df.loc[index, 'status_code'] = response.status_code
            
            except Exception as e:
                print(f"An error occurred for region {region_id}, commune {comuna_id}: {str(e)}")
                df.loc[index, 'status'] = 'Failed'
                continue  # Skip to the next iteration of the loop
            
    
    all_successful = all_successful and (df['status'] == 'Successful').all()

    if all_successful:
        break  # Exit the loop if all rows are successful
    else:
        attempts += 1
        time.sleep(DELAY_BETWEEN_ATTEMPTS)  # Delay between attempts

Attempt 1 of 5: 22 communes to process
 * Downloading report 1 out of 22
 * Downloading report 2 out of 22
 * Downloading report 3 out of 22
 * Downloading report 4 out of 22
 * Downloading report 5 out of 22
 * Downloading report 6 out of 22
 * Downloading report 8 out of 22
 * Downloading report 9 out of 22
 * Downloading report 10 out of 22
 * Downloading report 11 out of 22
 * Downloading report 13 out of 22
 * Downloading report 14 out of 22
 * Downloading report 15 out of 22
 * Downloading report 16 out of 22
 * Downloading report 17 out of 22
 * Downloading report 18 out of 22
 * Downloading report 19 out of 22
 * Downloading report 20 out of 22
 * Downloading report 21 out of 22
 * Downloading report 22 out of 22
 * Downloading report 23 out of 22
 * Downloading report 25 out of 22


# Drop specified columns
columns_to_drop = ['codigo_etiqueta', 'eventargument', 'viewstate']
out_df = df#.drop(columns=columns_to_drop).dropna().reset_index(drop=True)
out_df

In [13]:
#insert_unique_rows_from_dataframe(db_file_path, 'evals_html_downloaded_files', out_df, unique_columns=['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id'])